Deadline: 15.02.2022 23:59

# Домашние задание

1. Добавить Bias и посчитать для них градиенты.
2. Сравнить градинеты с тем, как считает PyTorch AutoGrad.

In [479]:
import numpy as np
import torch


In [480]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1


dtype = torch.float
device = torch.device("cpu")

In [481]:
# Create random input and output data
torch.manual_seed(42)

x = torch.randn(batch_size, input_size, device=device, dtype=dtype, requires_grad=True)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype, requires_grad=True)

# Randomly initialize weights
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)

# Randomly initialize bias
b1 = torch.randn(1, hidden_size, device=device, dtype=dtype, requires_grad=True)
b2 = torch.randn(1, output_size, device=device, dtype=dtype, requires_grad=True)


In [482]:
### Градиент ручками
h1 = torch.mm(x, w1) + b1
h1_relu = h1.clamp(min=0.)
y_pred = torch.mm(h1_relu, w2) + b2

loss = (y_pred - y).pow(2).sum()#.item()

with torch.no_grad():
    s = 2*(y_pred - y)
    h_grad = torch.mm(s, w2.T)
    h_grad[h1_relu==0] = 0

    grad_w2 = torch.mm(h1_relu.T, s)
    grad_b2 = torch.sum(s, axis=0)

    grad_w1 = torch.mm(x.T, h_grad)
    grad_b1 = torch.sum(h_grad, axis=0)

In [483]:
grad_b1

tensor([34.4378, 89.7073])

In [484]:
# Градиент ножками

loss.backward()
autograd_b1 = b1.grad
autograd_b2 = b2.grad

autograd_w1 = w1.grad
autograd_w2 = w2.grad

In [485]:
w1.grad

tensor([[-15.0927,  -9.1019],
        [ 45.8589, -99.3424],
        [-38.4661,   4.6612]])

In [486]:
autograd_w1

tensor([[-15.0927,  -9.1019],
        [ 45.8589, -99.3424],
        [-38.4661,   4.6612]])

#### Проверим равны ли ручные градиенты - пайторчовским

In [487]:
assert (grad_w1 == autograd_w1).all()
assert (grad_w2 == autograd_w2).all()
assert (grad_b1 == autograd_b1).all()
assert (grad_b2 == autograd_b2).all()

In [488]:
torch.manual_seed(42)

X = torch.randn(128, batch_size, input_size, device=device, dtype=dtype, requires_grad=True)
Y = torch.randn(128, batch_size, output_size, device=device, dtype=dtype, requires_grad=True)

In [489]:
w1_c = w1.clone().detach()
w2_c = w2.clone().detach()
b2_c = b2.clone().detach()
b1_c = b1.clone().detach()

In [490]:
b1

tensor([[-1.7379,  0.1841]], requires_grad=True)

### Запущу фиктивное обучение и буду выводить лосс, чтобы понять одинаково ли работает автоматический градиент торча и посчитанный руками

In [491]:
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
k = 0

for x,y in zip(X,Y):
    h1 = torch.mm(x, w1) + b1
    h1_relu = h1.clamp(min=0.)
    y_pred = torch.mm(h1_relu, w2) + b2
    k += 1
    loss = loss = (y_pred - y).pow(2).sum()#loss_fn(y_pred, y)
    if k % 10 == 0:
        print(k, loss.item())
    
    loss.backward()
   
    with torch.no_grad():
      
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        b1 -= learning_rate * b1.grad
        b2 -= learning_rate * b2.grad

        w1.grad.zero_()
        w2.grad.zero_()
        b1.grad.zero_()
        b2.grad.zero_()

10 205.3661346435547
20 204.7293701171875
30 286.1779479980469
40 226.7199249267578
50 251.77049255371094
60 195.32078552246094
70 164.57962036132812
80 186.96812438964844
90 166.11111450195312
100 184.751953125
110 197.72454833984375
120 222.17405700683594


In [493]:
# закостылю, чтобы не переписывать названия
w1, w2 = w1_c, w2_c
b1, b2 = b1_c, b2_c
k = 0 
with torch.no_grad():
  learning_rate = 1e-6
  for x,y in zip(X,Y):
      # Forward pass: compute predicted y
      h1 = torch.mm(x, w1) + b1
      h1_relu = h1.clamp(min=0.)
      y_pred = torch.mm(h1_relu, w2) + b2
      # Compute and print loss
      loss = (y_pred - y).pow(2).sum().item()
      k += 1
      if k % 10 == 0:
        print(k, loss)

      s = 2*(y_pred - y)
      h_grad = torch.mm(s, w2.T)
      h_grad[h1_relu==0] = 0

      grad_w2 = torch.mm(h1_relu.T, s)
      grad_b2 = torch.sum(s, axis=0)

      grad_w1 = torch.mm(x.T, h_grad)
      grad_b1 = torch.sum(h_grad, axis=0)

      # Update weights using gradient descent
      
      w1 -= learning_rate * grad_w1
      w2 -= learning_rate * grad_w2
      b1 -= learning_rate * grad_b1
      b2 -= learning_rate * grad_b2

10 205.43722534179688
20 204.7981414794922
30 286.26251220703125
40 226.7998504638672
50 251.86378479003906
60 195.39694213867188
70 164.6399688720703
80 187.0312042236328
90 166.1663055419922
100 184.79202270507812
110 197.80043029785156
120 222.251708984375


#### Вывод: Лосс одинаковый - градиент считается правильно